### Step 1. Importing Libraries

In [12]:
# Importing necessary libraries
import numpy as np  # for numerical operations
import pandas as pd  # for data manipulation
import random  # for shuffling the data
import nltk
import re  # for handling regular expressions

from nltk.stem import WordNetLemmatizer  # for lemmatizing words
from nltk.corpus import stopwords  # for stop word removal
from nltk.tokenize import word_tokenize  # for tokenizing sentences into words
nltk.download('punkt_tab')  # Downloads the 'punkt' tokenizer table used for tokenization of text into sentences or words

# Downloading necessary NLTK resources
nltk.download('stopwords')  # List of common stop words in English
nltk.download('punkt')  # Pre-trained tokenizer models
nltk.download('wordnet')  # WordNet lemmatizer dataset

# Libraries for text feature extraction and model training
from sklearn.feature_extraction.text import TfidfVectorizer  # Convert text into numerical features (TF-IDF)
from sklearn.linear_model import LogisticRegression  # Logistic regression for classification
from sklearn.svm import LinearSVC  # Support Vector Machines for classification

# Libraries for model evaluation
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix  # For model evaluation metrics
from sklearn.model_selection import KFold, cross_val_score  # For cross-validation

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/dianaterraza/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dianaterraza/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dianaterraza/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dianaterraza/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Step 2: Load & Prepare Dataset

The Sentence Polarity dataset contains 5,331 positive and 5,331 negative sentences. We'll load this dataset and prepare it for analysis.


In [13]:
%pip install kagglehub -q

Note: you may need to restart the kernel to use updated packages.


In [15]:
import kagglehub

# Download and get the path to the file
path = kagglehub.dataset_download('bricevergnou/spotify-recommendation', path='data.csv')

print("Path to dataset files:", path)

Path to dataset files: /Users/dianaterraza/.cache/kagglehub/datasets/bricevergnou/spotify-recommendation/versions/2/data.csv


In [16]:
# Read the positive and negative sentiment files
df_sent_pos = pd.read_csv('/Users/dianaterraza/Desktop/NLP/Data/sentence_polarity 2/rt-polarity.pos', sep='\t', header=None)  # Positive sentiment sentences
df_sent_neg = pd.read_csv('/Users/dianaterraza/Desktop/NLP/Data/sentence_polarity 2/rt-polarity.neg', sep='\t', header=None)  # Negative sentiment sentences

# Display the first few rows of the positive dataset to understand its structure
print(df_sent_pos.head())

                                                   0
0  the rock is destined to be the 21st century's ...
1  the gorgeously elaborate continuation of " the...
2                     effective but too-tepid biopic
3  if you sometimes like to go to the movies to h...
4  emerges as something rare , an issue movie tha...


### Step 4: Rename Columns

In [17]:
# Rename the column to 'sentence'
df_sent_pos.rename(columns={0: "sentence"}, inplace=True)
df_sent_neg.rename(columns={0: "sentence"}, inplace=True)

inplace=True ensures the changes are applied directly to the dataframes.

### Step 5: Data Preprocessing

1. Converts text to lowercase.
2. Removes punctuation using regular expressions.
3. Removes extra whitespace.
4. Tokenizes sentences into words.
5. Removes stop words.
6. Lemmatizes words.

In [18]:
# Define the preprocessing function
def preprocess_text(sentences):
    # Convert all tokens to lowercase
    sentences = [sentence.lower() for sentence in sentences]

    # Remove punctuation using regex
    sentences = [re.sub(r"[^\w\s]", "", sentence) for sentence in sentences]

    # Remove extra whitespaces between words
    sentences = [" ".join(sentence.split()) for sentence in sentences]

    # Tokenize sentences into words
    sentences = [word_tokenize(sentence) for sentence in sentences]

    # Remove stop words
    stop_words = set(stopwords.words('english'))  # Load English stop words
    filtered_sentences = []
    for sentence in sentences:
        filtered_sentence = [word for word in sentence if word not in stop_words]
        filtered_sentences.append(filtered_sentence)

    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentences = []
    for sentence in filtered_sentences:
        lemmatized_sentence = [lemmatizer.lemmatize(word) for word in sentence]
        lemmatized_sentences.append(lemmatized_sentence)

    return [' '.join(sentence) for sentence in lemmatized_sentences]

* Each preprocessing step cleans the text data and prepares it for model training.
* Stop words like "the", "is", and "and" are removed to focus on meaningful words.
* Lemmatization reduces words to their base form (e.g., "running" → "run").
* Note what we pass after the return keyword! [' '.join(sentence) for sentence in lemmatized_sentences] does an important step for us: it collects separate tokens back to a string. Here is a concrete example: we move from ['simplistic', 'silly', 'tedious'] to `
simplistic silly tedious. In other words, we move from a list of tokens to a single sentence in a form of a string.

### Step 6: Apply Preprocessing

In [19]:
# Preprocess the sentences
pos_preprocessed_sentences = preprocess_text(df_sent_pos['sentence'])
neg_preprocessed_sentences = preprocess_text(df_sent_neg['sentence'])

# Print the first preprocessed negative sentence
print(neg_preprocessed_sentences[0])

simplistic silly tedious


### Step 7: Combine Datasets

We merge the positive and negative sentences into a single list called sentences

In [20]:
# Combine preprocessed positive and negative sentences
sentences = pos_preprocessed_sentences + neg_preprocessed_sentences

### Step 8: Create Labels

Labels (also called targets) distinguish positive and negative sentences. Positive sentences are labeled as 1, and negative ones as 0.

In [21]:
# Create a list for all labels
polarities = []
polarities.extend([0] * len(df_sent_neg))  # Label negative sentences as 0
polarities.extend([1] * len(df_sent_pos))  # Label positive sentences as 1

The length of the polarities list matches the sentences list, maintaining the correct label for each sentence.

### Step 9: Shuffle Data

In [22]:
# Combine sentences and labels into a single list
combined = list(zip(sentences, polarities))

# Shuffle the combined list
random.shuffle(combined)

# Split the shuffled list back into sentences and labels
sentences[:], polarities[:] = zip(*combined)

### Step 10: Split Dataset

We’ll split the data into training and test sets, using 80% for training and 20% for testing.

In [23]:
# Define train-test split ratio
train_test_ratio = 0.8

# Calculate the size of the training set
train_set_size = int(train_test_ratio * len(sentences))

# Split data into training and test sets
X_train, X_test = sentences[:train_set_size], sentences[train_set_size:]
y_train, y_test = polarities[:train_set_size], polarities[train_set_size:]

# Print sizes of training and test sets
print("Size of training set:", len(X_train))
print("Size of test set:", len(X_test))

Size of training set: 8529
Size of test set: 2133


## **Vectorizing Texts, Training Models & Evaluating Their Performance**

### 1. Transforming Text into Features

TF-IDF stands for Term Frequency-Inverse Document Frequency 

The TF-IDF Vectorizer is a widely used tool for this purpose. It transforms sentences into a sparse matrix where each row corresponds to a document (sentence) and each column represents a term (word or token).

Here, we’ll use the TfidfVectorizer out of the box with its default configuration. By default, this includes:
* Only unigrams (individual terms/tokens) as features.
* A maximum document frequency of 1.0 (no terms are excluded based on frequency).
* Normalization applied to the resulting feature vectors.

In [24]:
# Import TF-IDF Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Define the vectorizer with default parameters
tfidf_vectorizer = TfidfVectorizer()

# Transform the training data into a TF-IDF matrix
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Check the number of samples and features
num_samples, num_features = X_train_tfidf.shape
print("#Samples: {}, #Features: {}".format(num_samples, num_features))

#Samples: 8529, #Features: 16490


### 2. Training the Classifier

Logistic Regression is a simple yet effective algorithm for binary classification tasks, such as predicting sentiment polarity.

In [25]:
# Import the Logistic Regression model
from sklearn.linear_model import LogisticRegression

# Train the Logistic Regression classifier
logistic_regression_classifier = LogisticRegression().fit(X_train_tfidf, y_train)

### 3. Evaluating the Classifier

After training the model, we need to assess its performance on unseen test data. Evaluation involves transforming the test data into the same TF-IDF format as the training data, making predictions, and calculating key metrics.

### 3.1 Transform Test Data

The general rule here is that .fit_transform() is always used for the training data and .transform() for any other data.

In [26]:
# Transform the test data into TF-IDF format
X_test_tfidf = tfidf_vectorizer.transform(X_test)

### 3.2 Predict Sentiment

In [27]:
# Predict polarities for the test data
y_pred = logistic_regression_classifier.predict(X_test_tfidf)

### 3.3 Generate Evaluation Report

We use the classification_report() function to generate precision, recall, and F1-score metrics for each class, as well as the overall accuracy.

In [28]:
# Import evaluation metrics
from sklearn.metrics import classification_report, accuracy_score

# Generate and display the classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.78      0.76      1055
           1       0.77      0.74      0.76      1078

    accuracy                           0.76      2133
   macro avg       0.76      0.76      0.76      2133
weighted avg       0.76      0.76      0.76      2133



### 4. Interpreting the Results
Let’s break down the key metrics:
* Precision: How many of the model’s positive predictions were correct? For example, the precision for class 0 (negative sentiment) is 75%, meaning 75% of the sentences predicted as negative were actually negative.

* Recall: How many of the actual positive cases were correctly identified? For class 1 (positive sentiment), the recall is 75%, meaning the model correctly identified 75% of all positive sentences.

* F1-score: This is the harmonic mean of precision and recall, providing a balanced measure. In this example, both classes have an F1-score of 0.75.

**Is 75% a Good F1-Score?**
A random classifier for a binary task would achieve an F1-score of ~50%.
Our model’s F1-score of 75% shows that it performs significantly better than random guessing. However, it’s not perfect, indicating room for improvement

## **Implementing K-Fold Cross-Validation**

In [29]:
# Import necessary library
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

# Perform 10-fold cross-validation on the training data
f1_scores_list = cross_val_score(
    LogisticRegression(),            # Model: Logistic Regression
    X_train_tfidf,                   # Features: TF-IDF transformed training data
    y_train,                         # Labels: Training labels
    cv=10,                           # Number of folds
    scoring="f1"                     # Evaluation metric: F1 score
)

# Display the F1 scores for each fold
print(f"F1 Scores for each fold: {f1_scores_list}")

# Calculate and display the mean and standard deviation of the F1 scores
print("F1 Score (Mean/Average): {:.3f}".format(f1_scores_list.mean()))
print("F1 Score (Standard Deviation): {:.3f}".format(f1_scores_list.std()))

F1 Scores for each fold: [0.75837321 0.75638051 0.73882353 0.74340528 0.72941176 0.77351916
 0.75059382 0.76091954 0.75919336 0.75417661]
F1 Score (Mean/Average): 0.752
F1 Score (Standard Deviation): 0.012


### Interpreting the Results
*1. F1 Scores Across Folds:*
* These represent the F1 scores obtained for each fold. In this example, the scores ranges from around 74 % to 77 %. They are fairly consistent, which is a good sign.

*2. Mean F1 Score:*
* The average F1 score is a reliable measure of the model’s overall performance.

*3. Standard Deviation:*
* A low standard deviation indicates that the model performs consistently across different subsets of the data.
* A high standard deviation might indicate issues with data shuffling or an insufficient dataset size.

### Key Points to Remember
* Use .fit_transform() only on the training data (or k-1 folds during cross-validation). For the test data or validation folds, always use .transform() to avoid data leakage.
* The test data is not involved in cross-validation. It remains unseen until final model evaluation.
* Choose the value of k based on dataset size and computational resources. Common choices are k=5 or k=10.

## **Perform Hyperparameter Tuning**

### A Hands-On Example of Hyperparameter Tuning
In this lesson, we'll perform a simple hyperparameter tuning exercise to identify the best combination of:

1. The classification algorithm (Logistic Regression VS LinearSVC)
2. The maximum size of n-grams (from 1 to 4)

### Step 1: Setting Up the Experiment:
We need to test every possible combination of our selected hyperparameters to identify which one yields the best F1-score during cross-validation.

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

# Initialize placeholders to store the best configuration
best_score = -1.0
best_classifier = None
best_ngram_size = -1

# Define the hyperparameters to test
classifiers = [LinearSVC(), LogisticRegression(solver="sag")]
ngram_sizes = [1, 2, 3, 4]

# Loop through all combinations of classifiers and n-gram sizes
for classifier in classifiers:
    for n in ngram_sizes:
        # Define the vectorizer with the current n-gram size
        vectorizer = TfidfVectorizer(ngram_range=(1, n))
        X_train_tfidf = vectorizer.fit_transform(X_train)  # Transform training data

        # Perform 10-fold cross-validation
        f1_scores = cross_val_score(classifier, X_train_tfidf, y_train, cv=10, scoring='f1')
        avg_f1_score = f1_scores.mean()  # Calculate average F1-score

        # Print the result for this combination
        print(f"Classifier: {type(classifier).__name__}, n-gram size: {n} => F1-score: {avg_f1_score:.3f}")

        # Save the best configuration
        if avg_f1_score > best_score:
            best_score = avg_f1_score
            best_classifier = classifier
            best_ngram_size = n

# Print the best configuration
print("\nBest Configuration:")
print(f"Classifier: {type(best_classifier).__name__}, Max n-gram size: {best_ngram_size}, F1-score: {best_score:.3f}")

Classifier: LinearSVC, n-gram size: 1 => F1-score: 0.742
Classifier: LinearSVC, n-gram size: 2 => F1-score: 0.755
Classifier: LinearSVC, n-gram size: 3 => F1-score: 0.755
Classifier: LinearSVC, n-gram size: 4 => F1-score: 0.754
Classifier: LogisticRegression, n-gram size: 1 => F1-score: 0.753
Classifier: LogisticRegression, n-gram size: 2 => F1-score: 0.747
Classifier: LogisticRegression, n-gram size: 3 => F1-score: 0.741
Classifier: LogisticRegression, n-gram size: 4 => F1-score: 0.738

Best Configuration:
Classifier: LinearSVC, Max n-gram size: 2, F1-score: 0.755


**LinearSVC Model (Support Vector Classifier with a Linear Kernel)**

LinearSVC is a Support Vector Machine (SVM) model that uses a linear kernel to classify data.

It is useful for high-dimensional datasets, such as text classification (NLP).

### Step 2: Training the Best Model
After identifying the best combination of parameters, we can train the final model on the entire training dataset and evaluate it using the unseen test data.

In [31]:
from sklearn.metrics import classification_report, accuracy_score

# Use the best configuration to train the final model
final_vectorizer = TfidfVectorizer(ngram_range=(1, best_ngram_size))
X_train_tfidf = final_vectorizer.fit_transform(X_train)
X_test_tfidf = final_vectorizer.transform(X_test)

best_classifier.fit(X_train_tfidf, y_train)
y_pred = best_classifier.predict(X_test_tfidf)

# Evaluate and display results
print("\nFinal Model Results:")
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred):.3f}")


Final Model Results:
              precision    recall  f1-score   support

           0       0.75      0.80      0.77      1055
           1       0.79      0.74      0.76      1078

    accuracy                           0.77      2133
   macro avg       0.77      0.77      0.77      2133
weighted avg       0.77      0.77      0.77      2133

Accuracy: 0.769
